<a href="https://colab.research.google.com/github/SORYNICI/ML/blob/main/HW_Korean_Emotion_Recognition%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 라이브러리 설치

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!pip install torch
!pip install transformers
!pip install numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 12.0 MB/s 
     |████████████████████████████████| 7.6 MB 57.4 MB/s 
     |████████████████████████████████| 163 kB 58.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
from tqdm import tqdm, tqdm_notebook
from transformers import AutoModel, AutoTokenizer, AdamW
from transformers import get_linear_schedule_with_warmup
import warnings
import os
warnings.filterwarnings('ignore')
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# 2. 데이터 로드

Task 1. 과제 Task 1에서 만든 파일을 본인의 구글 드라이브에 올린 후 파일의 URL을 Data URLs에 각각 맞춰 업데이트 해주세요.
기존에 적혀진 URL은 테스트를 위해 적은 양의 데이터만을 가진 것입니다.

In [7]:
# Task 1
# Data URLs
training_data_url = "https://drive.google.com/file/d/1EpopGE02nixNHc-y_bWhmGjVB1-Mmpqx/view?usp=drivesdk"
valid_data_url = "https://drive.google.com/file/d/1Ey9dCLSzhr_QJINSnpBj6pC5lC_4wqj9/view?usp=drivesdk"
test_data_url = "https://drive.google.com/file/d/1Eq3Pxy8lvZRL6nd-UX5CzxHMasq8zUM-/view?usp=drivesdk"

# Training Data
training_data_file_id = training_data_url.split("/")[-2] # /
!gdown $training_data_file_id

# Validation Data
valid_data_file_id = valid_data_url.split("/")[-2]
!gdown $valid_data_file_id

# Test Data
test_data_file_id = test_data_url.split("/")[-2]
!gdown $test_data_file_id

Downloading...
From: https://drive.google.com/uc?id=1EpopGE02nixNHc-y_bWhmGjVB1-Mmpqx
To: /content/train.csv
100% 15.0M/15.0M [00:00<00:00, 130MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Ey9dCLSzhr_QJINSnpBj6pC5lC_4wqj9
To: /content/valid.csv
100% 3.75M/3.75M [00:00<00:00, 66.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Eq3Pxy8lvZRL6nd-UX5CzxHMasq8zUM-
To: /content/test.csv
100% 2.34M/2.34M [00:00<00:00, 191MB/s]


In [8]:
train = pd.read_csv("train.csv")
valid = pd.read_csv("valid.csv")
test = pd.read_csv("test.csv")

In [9]:
print(train.shape)
print(valid.shape)
print(test.shape)

(180265, 2)
(45067, 2)
(28222, 2)


In [10]:
train.head()

,sentence,label
0,노후준비를 못 해서 후회되시는군요.,1
1,친구와 당분간 거리를 두기로 하셨군요.,2
2,남자친구가 바람을 피워서 많이 속상하시겠어요.,4
3,늙었다고 사람들이 무시하니 슬프네.,3
4,비 오는 날에 왜 가면 안 되는지 설득해야겠어.,0


In [11]:
device = torch.device("cuda:0")

# 3. 모델 정의

In [12]:
roberta = AutoModel.from_pretrained("klue/roberta-small")
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-small")

Downloading:   0%|          | 0.00/545 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/273M [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/roberta-small were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it f

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/248k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/752k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/173 [00:00<?, ?B/s]

Task 2. classifier에 맞는 내용을 코딩해주세요.

Hint: https://pytorch.org/docs/stable/generated/torch.nn.Linear.html 

In [13]:
class RoBERTaClassifier(nn.Module):
    def __init__(self, roberta, hidden_size=768, num_classes=2):
        super(RoBERTaClassifier, self).__init__()
        self.roberta = roberta
        # Task 2
        self.classifier = nn.Linear(hidden_size, num_classes)

    def forward(self, input_ids, attention_masks):
        outputs = self.roberta(input_ids=input_ids, attention_mask=attention_masks)
        return self.classifier(outputs.pooler_output)

In [14]:
model = RoBERTaClassifier(roberta=roberta).to(device)

# 4. 데이터셋 정의

Task 3. ``document`` 변수에 적합한 코드를 작성하세요.

Hint: ``self.labels`` 코드를 참고하세요.

In [15]:
class KERDataset(Dataset):
    def __init__(self, dataset, tokenizer):
        self.tokenizer = tokenizer
        self.dataset = dataset
        # Task 3
        document = self.dataset['sentence'].tolist()
        inputs = tokenizer(document, padding=True)
        self.input_ids = inputs['input_ids']
        self.attention_masks = inputs['attention_mask']
        self.labels = self.dataset['label'].tolist()

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return (self.input_ids[idx], self.attention_masks[idx], self.labels[idx])       

In [16]:
def collate_fn(batch):
    input_ids = [item[0] for item in batch]
    attention_masks = [item[1] for item in batch]
    labels = [item[2] for item in batch]
    return torch.LongTensor(input_ids), torch.LongTensor(attention_masks), torch.FloatTensor(labels)

In [17]:
train_ds = KERDataset(train, tokenizer)
valid_ds = KERDataset(valid, tokenizer)
test_ds = KERDataset(test, tokenizer)

In [18]:
batch_size = 32
warmup_ratio = 0.1
num_epochs = 5
log_interval = 400
learning_rate =  5e-5

In [19]:
train_dataloader = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, num_workers=5, collate_fn=collate_fn)
valid_dataloader = torch.utils.data.DataLoader(valid_ds, batch_size=batch_size, num_workers=5, collate_fn=collate_fn)
test_dataloader = torch.utils.data.DataLoader(test_ds, batch_size=batch_size, num_workers=5, collate_fn=collate_fn)

In [20]:
total_steps = len(train_dataloader) * num_epochs
warmup_step = int(total_steps * warmup_ratio)

In [21]:
optimizer = AdamW(model.parameters(), lr=learning_rate)
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = warmup_step,
                                            num_training_steps = total_steps)
loss_fn = nn.CrossEntropyLoss()

# 5. Training

In [22]:
def calc_accuracy(X,Y):
    correct = 0
    X, Y = X.tolist(), Y.tolist()
    for pred, label in zip(X, Y):
        if pred.index(max(pred)) == label.index(max(label)):
            correct += 1
    train_acc = correct/len(X)
    return train_acc

In [23]:
for e in range(num_epochs):
    train_acc = 0.0
    valid_acc = 0.0
    model.train()
    for batch_id, (input_ids, attention_masks, labels) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        input_ids = input_ids.to(device)
        attention_masks = attention_masks.to(device)
        out = model(input_ids=input_ids, attention_masks=attention_masks)
        labels = labels.tolist()
        labels = torch.FloatTensor([[0, 1] if l == 0 else [1, 0] for l in labels]).to(device)
        loss = loss_fn(out, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, labels)
        
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    model.eval()
    for batch_id, (input_ids, attention_masks, labels) in enumerate(tqdm_notebook(valid_dataloader)):
        input_ids = input_ids.to(device)
        attention_masks = attention_masks.to(device)
        labels = torch.FloatTensor([[0, 1] if l == 0 else [1, 0] for l in labels]).to(device)
        out = model(input_ids=input_ids, attention_masks=attention_masks)
        valid_acc += calc_accuracy(out, labels)
    print("epoch {} validation acc {}".format(e+1, valid_acc / (batch_id+1)))

  0%|          | 0/5634 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.6371586322784424 train acc 0.875
epoch 1 batch id 401 loss 0.36737731099128723 train acc 0.8358790523690773
epoch 1 batch id 801 loss 0.5174668431282043 train acc 0.8442181647940075
epoch 1 batch id 1201 loss 0.3624162971973419 train acc 0.8466382181515404
epoch 1 batch id 1601 loss 0.07926025986671448 train acc 0.8491177389131792
epoch 1 batch id 2001 loss 0.4222402572631836 train acc 0.850668415792104
epoch 1 batch id 2401 loss 0.35678932070732117 train acc 0.8517284464806331
epoch 1 batch id 2801 loss 0.44926154613494873 train acc 0.8526307568725455
epoch 1 batch id 3201 loss 0.28380894660949707 train acc 0.8541861918150578
epoch 1 batch id 3601 loss 0.4711858332157135 train acc 0.8546584282143849
epoch 1 batch id 4001 loss 0.40684252977371216 train acc 0.8553096100974756
epoch 1 batch id 4401 loss 0.2388964593410492 train acc 0.855224664848898
epoch 1 batch id 4801 loss 0.2826578617095947 train acc 0.855407727556759
epoch 1 batch id 5201 loss 0.20023638010

  0%|          | 0/1409 [00:00<?, ?it/s]

epoch 1 validation acc 0.8626020227111426


  0%|          | 0/5634 [00:00<?, ?it/s]

...

epoch 4 validation acc 0.8579001064584811


  0%|          | 0/5634 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.1227598562836647 train acc 0.9375
epoch 5 batch id 401 loss 0.2286396622657776 train acc 0.9159133416458853
epoch 5 batch id 801 loss 0.26785650849342346 train acc 0.912687265917603
epoch 5 batch id 1201 loss 0.1891927719116211 train acc 0.9151748542880933
epoch 5 batch id 1601 loss 0.041686322540044785 train acc 0.9160680824484697
epoch 5 batch id 2001 loss 0.4163898825645447 train acc 0.9163074712643678
epoch 5 batch id 2401 loss 0.12179487198591232 train acc 0.9172089754269055
epoch 5 batch id 2801 loss 0.3093407452106476 train acc 0.9183327383077472
epoch 5 batch id 3201 loss 0.2188132405281067 train acc 0.9196540143705092
epoch 5 batch id 3601 loss 0.25405406951904297 train acc 0.920082963065815
epoch 5 batch id 4001 loss 0.24498701095581055 train acc 0.9208557235691077
epoch 5 batch id 4401 loss 0.10330136120319366 train acc 0.9211258804817087
epoch 5 batch id 4801 loss 0.16115325689315796 train acc 0.9208302957717143
epoch 5 batch id 5201 loss 0.0996531

  0%|          | 0/1409 [00:00<?, ?it/s]

epoch 5 validation acc 0.8608075521001356


# 6. Test

In [24]:
test_acc = 0.0
model.eval()
for batch_id, (input_ids, attention_masks, labels) in enumerate(tqdm_notebook(test_dataloader)):
    input_ids = input_ids.to(device)
    attention_masks = attention_masks.to(device)
    labels = torch.FloatTensor([[0, 1] if l == 0 else [1, 0] for l in labels]).to(device)
    out = model(input_ids=input_ids, attention_masks=attention_masks)
    test_acc += calc_accuracy(out, labels)
print("Test acc : {}".format(test_acc / (batch_id+1)))

  0%|          | 0/882 [00:00<?, ?it/s]

Test acc : 0.8485284391534391


Task 4. 기존 코드에서 문제점/아쉬운 점들에 대해서 논하세요. 

해당 내용은 여기 코드 및 결과와 함께 보고서로 작성해서 제출하세요.

해당 문제점에 대한 해결 방안 및 이를 실제로 적용한 코드 작성 및 결과를 첨부한 경우 가산점이 있습니다.